# What's the terminal velocity of a skydiver?

## Names of group members

// put your names here!

## Goals of this assignment

The main goal of this assignment is to use numerical integration and differentiation to study the behavior of a skydiver.  You're going to use the numerical integration and differentiation techniques that you learned in the pre-class assignment.

## Some background knowledge that we need for this model

### Position, velocity, and acceleration 

In physics, three important properties of a moving object are its position ($\vec{x}$), velocity ($\vec{v}$), and acceleration ($\vec{a}$).  These are *vector quantities*, meaning that they have both a magnitude and a direction, and are related in the following way:

$\vec{v} = \frac{d\vec{x}}{dt}$ 

$\vec{a} = \frac{d\vec{v}}{dt}$ - i.e., acceleration is the rate of change of velocity (units of meters per second$^2$)

In words, velocity is the rate of change of position with time (and having units of length per time) and acceleration is the rate of change of velocity with time (and having units of length per time$^2$).  Given this, the fundamental theorem of calculus tells us that we can relate these quantities by integration as well.  Expressed mathematically:

$\vec{x} = \vec{x}_0 + \int_0^t \vec{v}(t) dt$

$\vec{v} = \vec{v}_0 + \int_0^t \vec{a}(t) dt$

So, we can get the position at any time by starting at the initial position and integrating the velocity over time, and can get the velocity at any time by starting with the initial velocity and integrating the acceleration over time.

### Terminal velocity

An object moving through a fluid like air or water experiences a force of friction - just think about what happens if you stick your hand out of the window of a moving car!  This is why airplanes need to run their engines constantly while in flight; when traveling at a constant speed, the force exerted by the engines just balances the force exerted by the air friction.  This force of friction always points in the *opposite direction* of the object's motion (in other words, in the opposite direction of its velocity).

A similar thing happens to a falling object.  As an object falls downward faster and faster, the force of gravity pulling downward is eventually perfectly balanced by the upward force from air resistance (upward because the direction of motion is down).  When these forces perfectly balance, the object experiences zero acceleration, and thus its velocity becomes constant.  We call this the **terminal velocity**.



## The challenge

Your professor happens to mention that he went skydiving last weekend.  He jumped from a stationary helicopter that was hovering 2,000 meters above the ground, and opened the parachute at the last possible moment.  In the interests of science, he had a friend stand on the ground with a radar gun and measure his velocity as a function of time.  This file, ```skydiver_time_velocities.csv```, has been provided to you to examine.  You are asked to do the following:

1.  Calculate and plot the position, velocity, and acceleration as a function of time.
2.  If you start the clock when your professor steps out of the helicopter (i.e., $t=0$), at what time does he land on the ground?
3.  At what time, and at what height above ground, does he reach terminal velocity?

In the cells below, we have provided two pieces of code: one that reads the data you want from the file into two Numpy arrays, and a second piece of code that can provide you with the velocity at any time.

In [ ]:
'''
The code in this cell opens up the file skydiver_time_velocities.csv 
and extracts two 1D numpy arrays of equal length.  One array is
of the velocity data taken by the radar gun, and the second is
the times that the data is taken.
'''

import numpy as np

skydiver_time, skydiver_velocity = np.loadtxt("skydiver_time_velocities.csv",
                                              delimiter=',',skiprows=1,unpack=True)


In [ ]:
'''
This is a piece of example code that shows you how to get the 
velocity at any time you want using the Numpy interp() method.  
This requires you to pick a time where you want the velocity
as an input parameter to the method, as well as the time and 
velocity arrays that you will interpolate from.
'''

time = 7.2  # time in seconds
vel = np.interp(time,skydiver_time,skydiver_velocity)
print("velocity at time {:.3f} s is {:.3f} m/s".format(time,vel))

In [ ]:
# This function computes the positions

def position(x0, times, velocities):
    x = np.zeros_like(times)
    I = x0
    x[0] = I
    for i in range(len(times)-1):
        delta_t = times[i+1] - times[i]
        I += velocities[i] * delta_t
        x[i+1] = I
        
    return x

In [ ]:
# This function computes the accelerations

def acceleration(times, velocities):    
    dft = np.zeros_like(times)
    for i in range(len(times)):
        if i == 0:
            dft[i] = (velocities[i] - velocities[i+1]) / (times[i] - times[i+1])
        elif i == (len(times) - 1):
            dft[i] = (velocities[i-1] - velocities[i]) / (times[i-1] - times[i])
        else:
            dft[i] = (velocities[i-1] - velocities[i+1]) / (times[i-1] - times[i+1])
            
    return dft

In [ ]:
# This is the main script for the position. 

# Modules
%matplotlib inline
import matplotlib.pyplot as plt

# Get position at every time
skydiver_position = position(2000, skydiver_time, skydiver_velocity)

# Plot the position as a function of time
plt.plot(skydiver_time, skydiver_position)
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Position as a function of time')

# The last measured position is above ground; interpolate to get time when the professor hits the ground
print('The last measured time is:', skydiver_time[3336])
print('The last measured position is:', skydiver_position[3336])
time_ground = np.interp(0, skydiver_position[::-1], skydiver_time[::-1])
print('The time on the ground is:', time_ground)

In [ ]:
# This is the main script for the acceleration

# Get acceleration at every time
skydiver_acceleration = acceleration(skydiver_time, skydiver_velocity)

# Plot the position as a function of time
plt.plot(skydiver_time, skydiver_acceleration)
plt.xlabel('Time')
plt.ylabel('Acceleration')
plt.title('Acceleration as a function of time')

# Interpolate to find when acceleration is zero (i.e., when professor reaches terminal velocity)
# Hmm this doesn't really work because the magnitude of the acceleration is always slightly above zero
# I suppose just look at the graphs to answer the terminal velocity question
time_terminal_velocity = np.interp(0, skydiver_acceleration, skydiver_time)
print('The time when the professor reaches terminal velocity is:', time_terminal_velocity)

In [ ]:
# height when the skydiver has basically reached terminal velocity
# by eye (using the graph produced immediately above) this happens around
# 22 seconds, give or take

height = np.interp(22,skydiver_time,skydiver_position)
print("the height when the skydiver reaches terminal velocity is:", height, 
      "meters above the ground")

## The second part of the challenge

In addition to your professor, a mouse and an elephant have also chosen to go skydiving.  (Choice may have had less to do with it than a tired physics professor trying to make a point; work with me here.)  Their speeds were recorded as well, in the files ```mouse_time_velocities.csv``` and ```elephant_time_velocities.csv```.  Read the data in for these two unfortunate creatures and store them in their own arrays. (Don't worry, they had parachutes too, they're just not very happy about it!)  Then, do the same calculations as before and plot the position, velocity, and acceleration as a function of time for all three individuals on the same set of graphs.  Do the mouse and/or elephant reach terminal velocity?  If so, at what time, and at what height above the ground?  

**ANSWER:**  The elephant never reaches terminal velocity; the mouse does it very quickly. 

In [ ]:
# put your code here!

mouse_time, mouse_velocity = np.loadtxt("mouse_time_velocities.csv",
                                              delimiter=',',skiprows=1,unpack=True)
mouse_position = position(2000, mouse_time, mouse_velocity)
mouse_acceleration = acceleration(mouse_time, mouse_velocity)

elephant_time, elephant_velocity = np.loadtxt("elephant_time_velocities.csv",
                                              delimiter=',',skiprows=1,unpack=True)
elephant_position = position(2000, elephant_time, elephant_velocity)
elephant_acceleration = acceleration(elephant_time, elephant_velocity)

person, mouse, elephant, = plt.plot(skydiver_time,skydiver_position,'k-',
                                    mouse_time,mouse_position,'g-',
                                    elephant_time,elephant_position,'b--')
plt.xlabel('time [s]')
plt.ylabel('position [m]')
plt.title("vertical height as a function of time")
plt.legend((person,mouse,elephant),('person','mouse','elephant'),loc='upper right')

In [ ]:
person, mouse, elephant, = plt.plot(skydiver_time,skydiver_acceleration,'k-',
                                    mouse_time,mouse_acceleration,'g-',
                                     elephant_time,elephant_acceleration,'b--')
plt.xlabel('time [s]')
plt.ylabel(r'acceleration [$m/s^2$]')
plt.title("vertical acceleration as a function of time")
plt.legend((person,mouse,elephant),('person','mouse','elephant'),loc='lower right')

In [ ]:
person, mouse, elephant, = plt.plot(skydiver_time,skydiver_velocity,'k-',
                                    mouse_time,mouse_velocity,'g-',
                                    elephant_time,elephant_velocity,'b--')
plt.xlabel('time [s]')
plt.ylabel('velocity [m/s]')
plt.title("vertical velocity as a function of time")
plt.legend((person,mouse,elephant),('person','mouse','elephant'),loc='lower right')


### Assignment wrapup

Please fill out the form that appears when you run the code below.  **You must completely fill this out in order to receive credit for the assignment!**

In [ ]:
from IPython.display import HTML
HTML(
"""
<iframe 
	src="https://goo.gl/forms/XvxmPrGnDOD3UZcI2?embedded=true" 
	width="80%" 
	height="1200px" 
	frameborder="0" 
	marginheight="0" 
	marginwidth="0">
	Loading...
</iframe>
"""
)

## Turn it in!

Turn this assignment in to the Day 19 dropbox in the "in-class activities" folder.